In [2]:
import os
from typing import List,Optional, Tuple


# Parameters
GROUP_ID: str = 'Group29'
ALGORITHM: str = 'bt'
PUZZLE_TYPE: str = 'easy'
PUZZLE_PATH: str = 'puzzles/escargot.txt'

EMPTY_VALUE: int = 0  # Note this value is somewhat arbitrary, but this seems to work well

class Board:
    def __init__(self):
        self.puzzle: List[List[int]] = [[EMPTY_VALUE]*9 for _ in range(9)]  # Stores the original puzzle
        self.state: List[List[int]] = [[EMPTY_VALUE]*9 for _ in range(9)]   # Stores the current board state

        """
        Its useful to store rows columns and squares inside "bitsets" where each value is a set of 9 booleans, one for each value (1...9).
        Then when checking for whether a number can be put in a cell, it is just bitwise or operations, which are very fast.
        """
        self.row_masks = [0]*9
        self.col_masks = [0]*9
        self.square_masks = [0]*9
        self.solved = False

        try:
            self.get_puzzle(PUZZLE_PATH)  # Pulls in the puzzle from a file
        except FileNotFoundError:
            print('Puzzle file not found')
            return

        for row in range(9):  # Copies the puzzle into the board state
            for col in range(9):
                self.set_cell(row, col, self.puzzle[row][col])

        match ALGORITHM:  # Selects the correct solver object
            case 'bt':
                self.solver: BacktrackingSolver = BacktrackingSolver(self)
            case 'fc':
                self.solver: ForwardCheckingSolver = ForwardCheckingSolver(self)
            case 'ac3':
                self.solver: AC3Solver = AC3Solver(self)
            case 'sa':
                self.solver: SimulatedAnnealingSolver = SimulatedAnnealingSolver(self)
            case 'ga':
                self.solver: GeneticSolver = GeneticSolver(self)
            case _:
                print('Unknown algorithm')
                return

        self.solver.solve()  # Runs the solver

        if self.check_solution():
            print("Solved!")
            print(f"Solution decision count: {self.solver.decision_count}")
            print(f"Solution stored at {self.output_puzzle_file()}")
            self.solved = True
        else:
            print("No solution found.")

    def __str__(self) -> str:
        """
        :return: The format is the same as the input file
        """
        out: List[str] = []
        for line in self.state:
            out.append(','.join(str(c) if c != EMPTY_VALUE else '?' for c in line))
        return '\n'.join(out) + '\n'

    def pretty_print(self):
        """
        Prints this board in a human-readable form, with seperated squares
        """
        print("-"*25)
        for r in range(9):
            if r % 3 == 0 and r != 0:
                print("|"+'+'.join(["-"*7 for _ in range(3)])+"|")  # horizontal separator

            row_str = "| "
            for c in range(9):
                if c % 3 == 0 and c != 0:
                    row_str += "| "
                val = self.state[r][c]
                row_str += (str(val) if val != EMPTY_VALUE else ".") + " "
            print(row_str.strip()+" |")
        print("-"*25)

    def get_cell(self, row: int, col: int) -> int:
        return self.state[row][col]

    def get_row(self, row: int) -> List[int]:
        return self.state[row]

    def get_col(self, col: int) -> List[int]:
        return [row[col] for row in self.state]

    def set_cell(self, row: int, col: int, value: int):
        self.state[row][col] = value
        if value != EMPTY_VALUE:
            bit = 1 << (value - 1)  # map 1..9 -> bits 0..8
            self.row_masks[row] |= bit
            self.col_masks[col] |= bit
            self.square_masks[(row//3)*3 + (col//3)] |= bit

    def clear_cell(self, row: int, col: int):
        original_value = self.state[row][col]
        if original_value != EMPTY_VALUE:
            bit = 1 << (original_value - 1)
            self.row_masks[row] &= ~bit
            self.col_masks[col] &= ~bit
            self.square_masks[(row//3)*3 + (col//3)] &= ~bit
        self.state[row][col] = EMPTY_VALUE

    def get_row_mask(self, row: int) -> int:
        return self.row_masks[row]

    def get_col_mask(self, col: int) -> int:
        return self.col_masks[col]

    def get_square_mask(self, row: int, col: int) -> int:
        return self.square_masks[(row//3)*3 + (col//3)]

    def get_cell_mask(self, row: int, col: int) -> int:
        cell_mask = self.get_row_mask(row)
        cell_mask |= self.get_col_mask(col)
        cell_mask |= self.get_square_mask(row, col)
        return cell_mask

    def get_puzzle(self, filename: str):
        """
        Throws fileNotFound exception
        :param filename: Takes in a file name of a puzzle file
        Sets this board to the puzzle given in the file
        """
        with open(filename) as f:
            for y, line in enumerate(f.readlines()):
                for x, value in enumerate(line.split(',')):
                    value = value.strip()
                    if not value.isnumeric():
                        self.puzzle[y][x] = EMPTY_VALUE
                    else:
                        self.puzzle[y][x] = int(value)

    def output_puzzle_file(self) -> str:
        puzzle_name = ""
        for ch in reversed(PUZZLE_PATH):
            if ch in ('/', '\\'):
                break
            puzzle_name = ch + puzzle_name
        puzzle_name = puzzle_name.split('.')[0]  # Removes txt extension

        os.makedirs('output', exist_ok=True)
        outfile_name = f'output/{GROUP_ID}_{ALGORITHM}_{PUZZLE_TYPE}_{puzzle_name}.txt'
        with open(outfile_name, 'w') as f:
            f.write(str(self))
        return outfile_name

    def check_solution(self) -> bool:
        """
        Separate checker from the solvers themselves, to ensure puzzles are correct.
        Not very optimized, but it doesn't really matter because it runs once per puzzle.
        :return: True is solved, False otherwise
        """
        # 1) No empties
        values: set[int] = set()
        for row in self.state:
            values.update(row)
        if EMPTY_VALUE in values:
            return False

        values = set()
        # 2) Rows and columns contain 9 distinct values (Sudoku assumes 1..9)
        for i in range(9):
            row = set(self.get_row(i))
            col = set(self.get_col(i))
            if len(row) != 9:
                return False
            if len(col) != 9:
                return False

            values = values.union(row)

            # 3) Each 3x3 square distinct
            r0 = (i // 3) * 3
            c0 = (i % 3) * 3
            square = [self.state[r][c] for r in range(r0, r0+3) for c in range(c0, c0+3)]
            if len(set(square)) != 9:
                return False

        # 4) Only legal values in the board (1...9)
        if len(values.intersection(set(range(1, 10)))) != 9:
            return False

        # 5) All original tiles from the puzzle are preserved
        for row in range(9):
            for col in range(9):
                if self.puzzle[row][col] != EMPTY_VALUE and self.state[row][col] != self.puzzle[row][col]:
                    return False

        return True

class BacktrackingSolver:
    def __init__(self, board: Board):
        self.decision_count: int = 0
        self.board: Board = board
        self.empty_squares: List[Tuple[int, int]] = [] # A list of all squares that need to be filled
        self.counted_numbers = {} # Counted values used in the puzzle so far, up to 9 of each (1...9) ignores empty squares
        self.get_all_empty()
        self.count_board_values()

    def get_all_empty(self):
        """
        Stores all empty squares into self.empty_squares
        """
        for row in range(9):
            for col in range(9):
                if self.board.state[row][col] == EMPTY_VALUE:
                    self.empty_squares.append((row, col))

    def count_board_values(self):
        """
        Count used values and store them in self.counted_numbers
        """
        self.counted_numbers = {i:0 for i in range(1, 10)}
        for row in range(9):
            for col in range(9):
                value = self.board.get_cell(row, col)
                if value != EMPTY_VALUE:
                    self.counted_numbers[value] += 1

    def sorted_values_by_use(self):
        return sorted(self.counted_numbers.items(),key=lambda x: x[1])

    def get_mrv(self) -> Tuple[int, int]:
        """
        mrv: most restricted value
        Look into the empty squares list, and finds the one with the most restrictions
        :return: Returns the mrv and pops it off the empty squares list
        """
        cell_index = -1
        best_restriction_count = -1

        for i, cell in enumerate(self.empty_squares):
            row, col = cell
            restriction_count = self.board.get_cell_mask(row, col).bit_count()

            if restriction_count > best_restriction_count:
                best_restriction_count = restriction_count
                cell_index = i
        return self.empty_squares.pop(cell_index)

    def solve(self) -> bool:
        """
        Solves the puzzle using backtracking. With a couple heuristics:
        1. mrv, start with whatever cell is the most restricted i.e. can only be a few values.
                This ensures that the DFS tree has as few starting branches as possible.
        2. least used value, start with the values that are least used in the puzzle so far:
                This limits the rest of the puzzle as minimally as possible.
        :return: Return the success of the solving algorithm
        """
        if len(self.empty_squares) == 0:
            return True

        row, col = self.get_mrv()
        for digit,_ in self.sorted_values_by_use():
            self.counted_numbers[digit] += 1
            self.decision_count += 1  # count each attempted assignment
            if self.is_digit_possible(row, col, digit):
                self.board.set_cell(row, col, digit)
                if self.solve():
                    return True
                self.board.clear_cell(row, col)
            self.counted_numbers[digit] -= 1
        self.empty_squares.append((row, col))
        return False

    def is_digit_possible(self, row: int, col: int, digit: int) -> bool:
        """
        Uses bitwise logic to quickly determine if a digit can go into a square.
        """
        bit = 1 << (digit - 1)
        return bool(bit & (~self.board.get_cell_mask(row, col)))

class ForwardCheckingSolver:
    def __init__(self, board: Board):
        self.decision_count: int = 0
        self.board: Board = board

    def solve(self) -> bool:
        pass

class AC3Solver:
    def __init__(self, board: Board):
        self.decision_count: int = 0
        self.board: Board = board

    def solve(self) -> bool:
        pass


class SimulatedAnnealingSolver:
    def __init__(self, board: Board):
        self.decision_count: int = 0
        self.board: Board = board

    def solve(self) -> bool:
        pass


class GeneticSolver:
    def __init__(self, board: Board):
        self.decision_count: int = 0
        self.board: Board = board

    def solve(self) -> bool:
        pass


def test_all():
    """
    Test all algorithms, with all puzzles. If any don't solve it terminates
    """
    global ALGORITHM, PUZZLE_PATH, PUZZLE_TYPE
    for puzzle in os.listdir("puzzles/"):
        for alg in ["bt"]:
            ALGORITHM = alg
            PUZZLE_PATH = f"puzzles/{puzzle}"
            if "Easy" in puzzle:
                PUZZLE_TYPE = "easy"
            elif "Med" in puzzle:
                PUZZLE_TYPE = "medium"
            elif "Hard" in puzzle:
                PUZZLE_TYPE = "hard"
            elif "Evil" in puzzle:
                PUZZLE_TYPE = "evil"
            else:
                PUZZLE_TYPE = "custom"
            b = Board()
            b.pretty_print()
            assert b.solved == True, f"NOT SOLVED: {puzzle} {alg}"
            print()


def main():
    Board().pretty_print()


if __name__ == '__main__':
    test_all()
    # main()


Solved!
Solution decision count: 163
Solution stored at output/Group29_bt_easy_Easy-P1.txt
-------------------------
| 1 9 8 | 7 5 6 | 2 4 3 |
| 7 5 4 | 2 8 3 | 6 1 9 |
| 2 6 3 | 1 9 4 | 8 5 7 |
|-------+-------+-------|
| 6 1 7 | 4 2 9 | 5 3 8 |
| 3 4 9 | 5 6 8 | 1 7 2 |
| 5 8 2 | 3 1 7 | 9 6 4 |
|-------+-------+-------|
| 8 3 5 | 6 4 2 | 7 9 1 |
| 4 2 1 | 9 7 5 | 3 8 6 |
| 9 7 6 | 8 3 1 | 4 2 5 |
-------------------------

Solved!
Solution decision count: 459
Solution stored at output/Group29_bt_easy_Easy-P2.txt
-------------------------
| 3 7 8 | 5 6 9 | 2 4 1 |
| 4 9 6 | 1 2 3 | 8 5 7 |
| 1 5 2 | 4 7 8 | 9 3 6 |
|-------+-------+-------|
| 2 8 3 | 7 9 6 | 5 1 4 |
| 9 6 1 | 8 4 5 | 7 2 3 |
| 7 4 5 | 2 3 1 | 6 9 8 |
|-------+-------+-------|
| 8 3 4 | 6 5 2 | 1 7 9 |
| 6 2 9 | 3 1 7 | 4 8 5 |
| 5 1 7 | 9 8 4 | 3 6 2 |
-------------------------

Solved!
Solution decision count: 172
Solution stored at output/Group29_bt_easy_Easy-P3.txt
-------------------------
| 1 3 2 | 5 6 9 | 4 8 7